In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### Obs: Esse código vai gerar arquivos de imagens na pasta em que está salvo.


## 1º Definir função para simular o circuito e retornar os gráficos

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  4 21:21:44 2020

@author: Edson Porto da Silva
"""
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import signal

def simulador(R,L,C,a,grafico):

    # Resolução numérica de um circuito RLC série alimentado por uma onda quadrada
    # de tensão.
    nome_arquivo = f'meu_grafico_{grafico}.jpg'
    # parâmetros do circuito:
    V0 = 0                # tensão inicial no capacitor [volts]
    I0 = 0                # corrente inicial no indutor [ampères]
    fq = 1               # frequência da onda quadrada [hertz]
    Vq = 15               # amplitude da onda quadrada [volts]
    w0 = 1/np.sqrt(L*C)   # frequência angular de ressonância [rad/segundo]

    t = np.arange(0,10,0.0001)*(2/10)   # discretização do intervalo de tempo [segundos]
    Vs = Vq*signal.square(2*np.pi*fq*t) # onda quadrada na entrada do circuito

    vC    = np.zeros(len(t))
    x     = np.zeros(len(t))

    # EDO da tensão sobre o capacitor: vc''(t)+(R/L)vc'(t)+vc(t)/LC = vs/LC

    # Solução numérica:
    vC[0] = V0       # condição incial de vc
    x[0]  = I0/C     # condição inicial da derivada vc'(t)
    #
    # Integração numérica via método de Euler:
    deltaT    = t[1]-t[0] # passo de integração
    numPoints = len(t)-1

    for kk in range(0, numPoints):
        vC[kk+1] = vC[kk]+x[kk]*deltaT                                # calcula vc(t+deltaT)
        x[kk+1]  = x[kk]+(-R/L*x[kk]-1/(L*C)*(vC[kk]-Vs[kk]))*deltaT  # calcula vc'(t+deltaT)

    # cálculo das tensões e da corrente partir de vc(t):
    i  = np.append(I0, C*np.diff(vC)/deltaT)    # corrente no circuito

    vR = R*i                                    # tensão no resistor
    vL = Vs-vR-vC                               # tensão no indutor(LKT)
    if a ==1:

        plt.figure(1, figsize=(10, 7), dpi=100, facecolor='w', edgecolor='k')
        plt.plot(t, i,color="black")
        #plt.legend(['corrente i(t)']);
        plt.grid(color='k', linestyle='--', linewidth=0.1)
        #plt.grid(False)
        #plt.ylabel('ampères (A)', fontsize = 14)
        #plt.xlabel('tempo (s)', fontsize = 14)
        #plt.title('Corrente no circuito RLC série', fontsize = 14)

        #Adicionado para tornar a imagem como a de um osciloscópio
        x_ticks = np.arange(0, 2.25, 0.25)  # Divisões no eixo x
        y_ticks = np.arange(-16, 16, 5)  # Divisões no eixo y
        plt.xticks(x_ticks)
        plt.yticks(y_ticks)
        plt.gca().set_xticklabels(['' for _ in x_ticks])  # Rótulos vazios
        plt.gca().set_yticklabels(['' for _ in y_ticks])  # Rótulos vazios
        #Se quiser as imagens no modelo antigo basta remover esse bloco
        
        
        #plt.savefig("C:\\Users\\ygorp\\Desktop\\Organizado\\Com as linhas\\Antes da normalização\\Imagens\\"+nome_arquivo, format='jpg')
        
        #Remove as bordas brancas que ficam nas imagens
        plt.savefig("C:\\Users\\ygorp\\Desktop\\Organizado\\Com as linhas\\Antes da normalização\\Imagens\\"+nome_arquivo, format='jpg', bbox_inches='tight')
        plt.close() # Não printar a imagem na tela

        plt.show(block=False)



## 2º Gerar os valores de R,L e C

In [ ]:

#CORRIGIR OS IFS
conjunto = np.empty((0,3)) #Matriz que armazena os valores de R,L e C
tam = 0 #Quantidade de valores armazenados (linhas da matriz)
w_arm = np.empty((0,))
while tam < 2000:    #Superamortecido

    l = random.uniform(0.01, 0.1)
    c = random.uniform(0.01, 0.1)
    r = random.uniform(1, 20)
    w = 1/(l*c)**(1/2)
    alpha = r/(2*l)
    param= abs(w-alpha)
    if alpha>w and w>10 and w<100 :
        w_arm = np.hstack((w_arm,w))
        conj =r,l,c
        conjunto =np.vstack((conjunto,conj))
        tam+=1

while tam < 4000:     #Subamortecido

    l = random.uniform(0.01, 0.1)
    c = random.uniform(0.01, 0.1)
    r = random.uniform(1, 20)
    w = 1/(l*c)**(1/2)
    alpha = r/(2*l)
    param= abs(w-alpha)
    if alpha<w and w>10 and w<100 :
        w_arm = np.hstack((w_arm,w))
        conj =r,l,c
        conjunto =np.vstack((conjunto,conj))
        tam+=1
while tam < 6000:     #Criticamente amortecido

    l = random.uniform(0.01, 0.1)
    c = random.uniform(0.01, 0.1)
    r = random.uniform(1, 20)
    w = 1/(l*c)**(1/2)
    alpha = r/(2*l)
    param= abs(w-alpha)
    if param<0.001 and w>10 and w<100 :
        w_arm = np.hstack((w_arm,w))
        conj =r,l,c
        conjunto =np.vstack((conjunto,conj))
        tam+=1


### 3º Avaliar os dados

In [ ]:
print("Valor máximo de w: ",np.max(w_arm))
print("Valor mínimo de w: ",np.min(w_arm))
print("Valor médio de w: ",np.mean(w_arm))

print("")
print("Valor máximo de resistor: ",np.max(conjunto[:,0]))
print("Valor mínimo de resistor: ",np.min(conjunto[:,0]))
print("Valor médio de resistor: ",np.mean(conjunto[:,0]))

print("")
print("Valor máximo de indutor: ",np.max(conjunto[:,1]))
print("Valor mínimo de indutor: ",np.min(conjunto[:,1]))
print("Valor médio de indutor: ",np.mean(conjunto[:,1]))

print("")
print("Valor máximo de capacitor: ",np.max(conjunto[:,2]))
print("Valor mínimo de capacitor: ",np.min(conjunto[:,2]))
print("Valor médio de capacitor: ",np.mean(conjunto[:,2]))
print(w_arm.shape)


## SEPARAR O CONJUNTO DE TREINAMENTO E DE TESTE

In [ ]:
np.random.shuffle(conjunto)
conjunto_treinamento = conjunto[0:5000,:]
conjunto_teste = conjunto[5000:,:]

print(conjunto_treinamento.shape)
print(conjunto_teste.shape)

## Vizualizar os gráficos e salvar os conjuntos de teste e treinamento

In [ ]:
m = conjunto.shape[0]

for i in range (m):
    R=conjunto[i,0]
    L=conjunto[i,1]
    C=conjunto[i,2]
    #print("Gráfico número:",i,"\n")
    simulador(R,L,C,1,i)


In [ ]:
np.save("C:\\Users\\ygorp\\Desktop\\Organizado\\Com as linhas\\Antes da normalização\\Imagens\\conjunto_treinamento.npy",conjunto_treinamento)
np.save("C:\\Users\\ygorp\\Desktop\\Organizado\\Com as linhas\\Antes da normalização\\Imagens\\conjunto_teste.npy",conjunto_teste)
